In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Start

In [ ]:
import sys
sys.path.append("..") 
import torchvision.transforms as ttfs

import torch, PIL
import torch.nn as nn, torch.nn.functional as F, numpy as np, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from torch import tensor
from tqdm import tnrange as trange, tqdm_notebook as tqdm

In [ ]:
def pil2tensor(image):
    arr = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))
    arr = arr.view(image.size[1], image.size[0], -1)
    arr = arr.permute(2,0,1)
    return arr.float().div_(255)

In [ ]:
class ImageDataset1(Dataset):
    def __init__(self, fns, labels, classes=None):
        if classes is None: classes = list(set(labels))
        self.classes = classes
        self.class2idx = {v:k for k,v in enumerate(classes)}
        self.fns = np.array(fns)
        self.y = [self.class2idx[o] for o in labels]
        
    @classmethod
    def from_folder(cls, folder, classes=None, test_pct=0.):
        if classes is None: classes = [cls.name for cls in find_classes(folder)]
            
        fns,labels = [],[]
        for cl in classes:
            fnames = get_image_files(folder/cl)
            fns += fnames
            labels += [cl] * len(fnames)
            
        if test_pct==0.: return cls(fns, labels, classes=classes)
        fns,labels = np.array(fns),np.array(labels)
        is_test = np.random.uniform(size=(len(fns),)) < test_pct
        return (cls(fns[~is_test], labels[~is_test], classes=classes),
            cls(fns[is_test], labels[is_test], classes=classes))

    def __len__(self): return len(self.fns)

    def __getitem__(self,i):
        x = PIL.Image.open(self.fns[i]).convert('RGB')
        x = ttfs.Resize((sz,sz))(x)
        x = pil2tensor(x)*2-1
        return x,self.y[i]

In [ ]:
def get_image_files(c):
    return [o for o in list(c.iterdir())
            if not o.name.startswith('.') and not o.is_dir()]

In [ ]:
sz = 224
DATA_PATH = Path('../data')
PATH = DATA_PATH/'caltech101'

data_mean,data_std = map(tensor, ([0.5355,0.5430,0.5280], [0.2909,0.2788,0.2979]))

classes = ['airplanes','Motorbikes','Faces','watch','Leopards']
np.random.seed(42)
train_ds,valid_ds = ImageDataset1.from_folder(PATH, test_pct=0.2, classes=classes)
# train_ds = ImageDataset1.from_folder(PATH, classes=classes)
# valid_ds = ImageDataset1.from_folder(PATH, classes=classes)
classes = train_ds.classes

default_device = torch.device('cuda', 0)
c = len(classes)
c,len(train_ds),len(valid_ds)

In [ ]:
class CudaDataLoader():
    def __init__(self,dl): self.dl = dl
    def __len__(self): return len(self.dl)
    def __iter__(self):
        for o in self.dl: yield o[0].cuda(), o[1].cuda()

In [ ]:
train_dl = CudaDataLoader(DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=8))
valid_dl = CudaDataLoader(DataLoader(valid_ds, batch_size=2*64, shuffle=False, num_workers=8))

## Train

In [ ]:
from collections import Iterable

def listify(p=None, q=None):
    "Makes p a list that looks like q"
    if p is None: p=[]
    elif not isinstance(p, Iterable): p=[p]
    n = q if type(q)==int else 1 if q is None else len(q)
    if len(p)==1: p = p * n
    return p

class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func=func
        
    def forward(self, x): return self.func(x)

def ResizeBatch(*size): return Lambda(lambda x: x.view((-1,)+size))
def Flatten(): return Lambda(lambda x: x.view((x.size(0), -1)))
def PoolFlatten(): return nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten())

In [ ]:
def simple_cnn(n_classes, actns, kernel_szs, strides, bn=False):
    kernel_szs = listify(kernel_szs, len(actns)-1)
    strides    = listify(strides   , len(actns)-1)
    layers = [conv2_relu(actns[i], actns[i+1], kernel_szs[i], stride=strides[i], bn=bn)
        for i in range(len(strides))]
    layers += [PoolFlatten(), nn.Linear(actns[-1], n_classes)]
    return nn.Sequential(*layers)

def conv2_relu(nif, nof, ks, stride, bn=False):
    layers = [nn.Conv2d(nif, nof, ks, stride, padding=ks//2), nn.ReLU()]
    if bn: layers.append(nn.BatchNorm2d(nof))
    return nn.Sequential(*layers)

In [ ]:
def get_model(): return simple_cnn(c, [3,16,16,16], 3, 2, bn=False)

def loss_batch(model, xb, yb, loss_fn, opt=None):
    loss = loss_fn(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

def fit(epochs, model, loss_fn, opt, train_dl, valid_dl):
    for epoch in trange(epochs):
        model.train()
        for xb,yb in train_dl:
            loss,_ = loss_batch(model, xb, yb, loss_fn, opt)

        model.eval()
        with torch.no_grad():
            losses,nums = zip(*[loss_batch(model, xb, yb, loss_fn)
                                for xb,yb in valid_dl])
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

        print(epoch, val_loss)

## Here

In [ ]:
model = get_model().cuda()
opt = optim.Adam(model.parameters(), 1e-3, betas=(0.9,0.99), weight_decay=1e-5)

In [ ]:
fit(2, model, F.cross_entropy, opt, train_dl, valid_dl)

In [ ]:
model.eval()
with torch.no_grad():
    *val_metrics,nums = zip(*[loss_batch(model, xb, yb, F.cross_entropy)
                                for xb,yb in valid_dl])
    val_metrics = [np.sum(np.multiply(val,nums)) / np.sum(nums) for val in val_metrics]

val_metrics

In [ ]:
torch.save(model.state_dict(),PATH/'model1.pt')

## Eval

In [ ]:
model1 = get_model().cuda()

In [ ]:
model1.load_state_dict(torch.load(PATH/'model1.pt'))

In [ ]:
model1.eval()
with torch.no_grad():
    *val_metrics,nums = zip(*[loss_batch(model1, xb, yb, F.cross_entropy)
                                for xb,yb in valid_dl])
    val_metrics = [np.sum(np.multiply(val,nums)) / np.sum(nums) for val in val_metrics]

val_metrics

In [ ]:
model1.eval()
with torch.no_grad():
    *val_metrics,nums = zip(*[loss_batch(model1, xb, yb, F.cross_entropy)
                                for xb,yb in valid_dl])
    val_metrics = [np.sum(np.multiply(val,nums)) / np.sum(nums) for val in val_metrics]

val_metrics